In [1]:
import sqlite3
import requests
import json
import csv

# NBA Feed API URL Endpoints

In [2]:
base_url = "https://data.nba.com/data/v2022/json/mobile_teams/nba"

base_url_2022 = base_url + "/2022"
# Teams
teams_info_url = base_url_2022 + "/teams/00_team_info.json"
# Players
players_info_url = base_url_2022 + "/players/00_player_info.json"
# Games
games_info_url = base_url_2022 + "/league/00_full_schedule.json"
# Game Lineup
def get_game_lineup_url(game_id: str):
    return base_url_2022 + f'''/scores/roster_lineup/{game_id}_roster_lineup.json'''
# Team Roster
def get_team_schedule_url(team_name: str):
    return base_url_2022 + f'''/teams/{team_name}_roster.json'''
# Box Score For Game
def get_box_score_for_game_url(game_id: str):
    return base_url_2022 + f'''/scores/gamedetail/{game_id}_gamedetail.json'''
# Team Games Schedule, not necessarily will be needed.
def get_team_schedule_url(team_name: str):
    return base_url_2022 + f'''/teams/{team_name}_schedule.json'''

In [3]:
# Request according to the url
def request_to_nba_feed(url: str):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}') 
    except requests.exceptions.ConnectionError as conn_err:
        print(f'Error Connecting: {conn_err}') 
    except requests.exceptions.Timeout as time_err:
        print(f'Timeout Error: {time_err}') 
    except requests.exceptions.RequestException as req_err:
        print(f'Something went wrong: {req_err}') 


# Utils

In [4]:
def get_percentage(numerator, denominator) -> int:
    if(denominator == 0 or  numerator == 0):
        return 0
    else:
        return int((numerator / denominator) * 100)

In [5]:
def save_into_csv(file_name, elements):
    # Get the headers (property names)
    headers = [prop for prop in dir(elements[0]) if not callable(getattr(elements[0], prop)) and not prop.startswith("__")]

    with open(f'{file_name}.csv', "w", newline="") as f:
        csv_writer = csv.writer(f)
        # Write headers
        csv_writer.writerow(headers)
        # Iterate and save into csv
        for element in elements:
            csv_writer.writerow([getattr(element, prop) for prop in headers])

# Class objects:

In [6]:
class Team:
    def __init__(self, ID, TeamName, TeamCode, City):
        self.ID = ID
        self.TeamName = TeamName
        self.TeamCode = TeamCode
        self.City = City  
        
    def __str__(self):
        return f'{self.ID} {self.TeamName} {self.TeamCode} {self.City}'

In [7]:
class Player:
    def __init__(self, ID, FirstName, LastName, DateOfBirth, Position, Height, Weight, YearsInLeague, Country):
        self.ID = ID
        self.FirstName = FirstName
        self.LastName = LastName
        self.DateOfBirth = DateOfBirth
        self.Position = Position
        self.Height = Height
        self.Weight = Weight
        self.YearsInLeague = YearsInLeague
        self.Country = Country
        
    def __str__(self):
        return f"ID: {self.ID}, Name: {self.FirstName} {self.LastName}, DOB: {self.DateOfBirth}, Position: {self.Position}, Height: {self.Height}, Weight: {self.Weight}, Years in League: {self.YearsInLeague}, Country: {self.Country}"

In [8]:
class TeamRoster:
    def __init__(self, TeamID, PlayerID):
        self.TeamID = TeamID
        self.PlayerID = PlayerID
        
    def __str__(self):
        return f'{self.TeamID} {self.PlayerID}'

In [9]:
class Game:
    def __init__(self, ID, Date, HomeTeamID, VisitorTeamID, LiveStatus):
        self.ID = ID
        self.Date = Date
        self.HomeTeamID = HomeTeamID
        self.VisitorTeamID = VisitorTeamID
        self.LiveStatus = LiveStatus
        
    def __str__(self):
        return f'{self.ID} {self.Date} {self.HomeTeamID} {self.VisitorTeamID} {self.LiveStatus}'

In [10]:
class GameLineUp:    
    def __init__(self, GameID, TeamID, PlayerID):
        self.GameID = GameID
        self.TeamID = TeamID
        self.PlayerID = PlayerID
        
    def __str__(self):
        return f'{self.GameID} {self.TeamID} {self.PlayerID}'

In [11]:
class GameScore:
    def __init__(self, GameID, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TournOvers, PTS):
        self.GameID = GameID
        self.FGA = FGA
        self.FGM = FGM
        self.FGPER = FGPER
        self.ThreePA = ThreePA
        self.ThreePM = ThreePM
        self.ThreePPER = ThreePPER
        self.FTA = FTA
        self.FTM = FTM
        self.FTPER = FTPER
        self.OREB = OREB
        self.DREB = DREB
        self.AST = AST
        self.STL = STL
        self.BLK = BLK
        self.TournOvers = TournOvers
        self.PTS = PTS
        
    def __str__(self):
        return f'GameID: {self.GameID}, FGA: {self.FGA}, FGM: {self.FGM}, FGPER: {self.FGPER}, ThreePM: {self.ThreePM}, ThreePA: {self.ThreePA}, ThreePPER: {self.ThreePPER}, FTM: {self.FTM}, FTA: {self.FTA}, FTPER: {self.FTPER}, OREB: {self.OREB}, DREB: {self.DREB}, AST: {self.AST}, STL: {self.STL}, BLK: {self.BLK}, TournOvers: {self.TournOvers}, PTS: {self.PTS}'

In [12]:
class GameTeamScore(GameScore):
    def __init__(self, GameID, TeamID, ScoreQOne, ScoreQTwo, ScoreQThree, ScoreQFour, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TournOvers, PTS):
        self.TeamID = TeamID
        self.ScoreQOne = ScoreQOne
        self.ScoreQTwo = ScoreQTwo
        self.ScoreQThree = ScoreQThree
        self.ScoreQFour = ScoreQFour
        super().__init__(GameID, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TournOvers, PTS)

    def __str__(self):
        return f'''TeamID: {self.TeamID}, Q1: {self.ScoreQOne}, Q2: {self.ScoreQTwo}, Q3: {self.ScoreQThree}, Q4: {self.ScoreQFour}, {super().__str__()}'''

In [13]:
class GamePlayerScore(GameScore):
    def __init__(self, GameID, PlayerID, PlayTimeInSec, PlusMinusRank, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TournOvers, PTS):
        self.PlayerID = PlayerID
        self.PlayTimeInSec = PlayTimeInSec
        self.PlusMinusRank = PlusMinusRank
        super().__init__(GameID, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TournOvers, PTS)
        

    def __str__(self):
        return f'''PlayerID: {self.PlayerID}, PlayTimeInSec: {self.PlayTimeInSec}, PlusMinusRank: {self.PlusMinusRank}, {super().__str__()}'''

# Parsers

In [14]:
# Team Parser
def team_parser(raw_data) -> list[Team]:

    # Access the 't' list within the json
    teams_data = raw_data['data']['tms']['t']

    # Initialize an empty list to hold the teams
    teams_list = []

    # Iterate through each team in the teams_data list
    for team in teams_data:
        # Extract the relevant information from the json and create a new Teams object
        id = team['tid']
        name = team['tn']
        code = team['ta']
        city = team['tc']
        teams_list.append(Team(id, name, code, city))

    return teams_list

In [15]:
# Player Parser
def player_parser(raw_data) -> list[Player]:
    
    # Access the 'pl' list within the json
    players_data = raw_data['data']['pls']['pl']

    # Initialize an empty list to hold the players
    players_list = []

    # Iterate through each player in the players_data list
    for player in players_data:
        # Extract the relevant information from the json and create a new Players object
        id = player['pid']
        first_name = player['fn']
        last_name = player['ln']
        date_of_birth = player['dob']
        position = player['pos']
        height =  player['ht']
        weight =  player['wt']
        years_in_league =  player['y']
        country =  player['co']
        players_list.append(Player(id, first_name, last_name, date_of_birth, position, height, weight, years_in_league, country))
    
    return players_list

In [16]:
# Team Roster Parser
def team_roster_parser(raw_data) -> list[TeamRoster]:
    
    # Access the 'pl' list within the json
    players_data = raw_data['data']['pls']['pl']

    # Initialize an empty list to hold the players
    team_roster_list = []

    # Iterate through each player in the players_data list
    for player in players_data:
        # Extract the relevant information from the json and create a new TeamRoster object
        team_id = player['tid']
        player_id = player['pid']
        team_roster_list.append(TeamRoster(team_id, player_id))

    return team_roster_list

In [17]:
# Game Parser
def game_parser(raw_data) -> list[Game]:
    
    # Access the 'lscd' list within the json
    games_data = raw_data['data']['lscd']

    # Initialize an empty list to hold the players
    games_list = []

    # Iterate through each month in the games_data list
    for games_month in games_data:
        game_data_raw = games_month['mscd']['g']
        for game in game_data_raw:
            # Extract the relevant information from the json and create a new Game object
            id = game['gid']
            date = game['gdtutc']
            home_team_id = game['h']['tid']
            visitors_team_id = game['v']['tid']
            live_status = game['stt']
            games_list.append(Game(id, date, home_team_id, visitors_team_id, live_status))

    return games_list

In [18]:
# Game LineUp Parser
def game_lineup_parser(raw_data) -> list[GameLineUp]:
    
    # Access the 'data' list within the json
    lineups_teams_data = raw_data['data']
    #Extract game_id
    game_id = lineups_teams_data['gameId']
    
    # Initialize an empty list to hold the players
    game_lineup_list = []
    
    #Extract only the home team
    home_players_lineups = lineups_teams_data['home']
    team_lineup_id = home_players_lineups['teamId']
    #Iterate only on the players
    for home_player_lineup in home_players_lineups['players']:
        team_id = team_lineup_id
        player_id = home_player_lineup['personId']
        game_lineup_list.append(GameLineUp(game_id, team_id, player_id))
        
    #Extract only the visitor team
    visitor_players_lineups = lineups_teams_data['visitor']
    team_lineup_id = visitor_players_lineups['teamId']
    #Iterate only on the players
    for home_player_lineup in visitor_players_lineups['players']:
        # Extract the relevant information from the json and create a new TeamRoster object
        team_id = team_lineup_id
        player_id = home_player_lineup['personId']
        game_lineup_list.append(GameLineUp(game_id, team_id, player_id))
    
    return game_lineup_list

In [19]:
# Game Box Score Parser
def game_boxscore_parser(raw_data) -> (list[GameTeamScore], list[GamePlayerScore]):
    
    # Access the 'data' list within the json
    boxscore_data = raw_data['data']['g']
    # Extract game_id
    game_id = boxscore_data['gid']
    
    # Initialize an empty lists to hold the scores
    game_teams_score_list = []
    game_players_score_list = []
    
    # Extract visitor team score
    visitor_scores = boxscore_data['vls']
    # called the team_score_helper Func to parse the relevent fields
    game_teams_score_list.append(game_team_score_parser_helper(visitor_scores, game_id))
    
    # Iterate over the players stats and extract the relevent stat with the game_player_score Func
    for player_stats in visitor_scores['pstsg']:
        game_players_score_list.append(game_player_score_parser_helper(player_stats, game_id))
    
    # Extract Home team score
    home_scores = boxscore_data['hls']
    # called the team_score_helper Func to parse the relevent fields
    game_teams_score_list.append(game_team_score_parser_helper(home_scores, game_id))
    
    # Iterate over the players stats and extract the relevent stat with the game_player_score Func
    for player_stats in home_scores['pstsg']:
        game_players_score_list.append(game_player_score_parser_helper(player_stats, game_id))

    return (game_teams_score_list, game_players_score_list)

# Game Team Score Parser helper
def game_team_score_parser_helper(team_score_json, game_id) -> GameTeamScore:
    team_points = team_score_json['s']
    team_id = team_score_json['tid'] 
    q1 = team_score_json['q1']
    q2 = team_score_json['q2']
    q3 = team_score_json['q3']
    q4 = team_score_json['q4']
        
    team_stat = team_score_json['tstsg']
    fga = team_stat['fga']
    fgm = team_stat['fgm']   
    fg_per = get_percentage(team_stat['fgm'], team_stat['fga'])
    three_pa = team_stat['tpa']
    three_pm = team_stat['tpm']
    three_per = get_percentage(team_stat['tpm'], team_stat['tpa'])
    fta = team_stat['fta']
    ftm = team_stat['ftm']
    ft_per = get_percentage(team_stat['ftm'], team_stat['fta'])
    o_reb = team_stat['oreb']
    d_reb = team_stat['dreb']
    ast = team_stat['ast']
    stl = team_stat['stl']
    blk = team_stat['blk']
    to = team_stat['tov']
    pts = team_points       
    
    return GameTeamScore(game_id, team_id, q1, q2, q3, q4, fga, fgm, fg_per, three_pa, three_pm, three_per, fta, ftm, ft_per, o_reb, d_reb, ast, stl, blk, to, pts)

# Game Player Score Parser helper
def game_player_score_parser_helper(player_stat, game_id) -> GamePlayerScore:
    
    fga = player_stat['fga']
    fgm = player_stat['fgm']   
    fg_per = get_percentage(player_stat['fgm'], player_stat['fga'])
    three_pa = player_stat['tpa']
    three_pm = player_stat['tpm']
    three_per = get_percentage(player_stat['tpm'], player_stat['tpa'])
    fta = player_stat['fta']
    ftm = player_stat['ftm']
    ft_per = get_percentage(player_stat['ftm'], player_stat['fta'])
    o_reb = player_stat['oreb']
    d_reb = player_stat['dreb']
    ast = player_stat['ast']
    stl = player_stat['stl']
    blk = player_stat['blk']
    to = player_stat['tov']
    pts = player_stat['pts']
    player_id = player_stat['pid']
    play_time_in_sec = player_stat['totsec']
    plus_minus_rank = player_stat['pm']
    
    return GamePlayerScore(game_id, player_id, play_time_in_sec, plus_minus_rank, fga, fgm, fg_per, three_pa, three_pm, three_per, fta, ftm, ft_per, o_reb, d_reb, ast, stl, blk, to, pts)

# Initialization

In [20]:
#Initialize Teams
all_teams = team_parser(request_to_nba_feed(teams_info_url))

In [21]:
#Initialize Players
all_players = player_parser(request_to_nba_feed(players_info_url))

In [22]:
#Initialize Games
all_games = game_parser(request_to_nba_feed(games_info_url))

In [23]:
#Initialize Rosters
all_team_rosters = team_roster_parser(request_to_nba_feed(players_info_url))

In [124]:
#game example
game_id = '0012200005'

In [25]:
game_lineup = game_lineup_parser(request_to_nba_feed(get_game_lineup_url(game_id)))

In [26]:
game_team_scores, game_player_scores = game_boxscore_parser(request_to_nba_feed(get_box_score_for_game_url(game_id)))

In [27]:
#Initialize Lineups and scores(Teams and Players) for the the 'Finish' games
all_game_lineups = []
all_game_team_scores = []
all_game_player_scores = []

final_games_ids = [game.ID for game in all_games if game.LiveStatus == 'Final']

num_of_iterations = len(final_games_ids)
counter = 0
for final_game_id in final_games_ids:
    counter += 1
    print(f'{counter} from {num_of_iterations}')
    all_game_lineups.extend(game_lineup_parser(request_to_nba_feed(get_game_lineup_url(final_game_id))))
    
    game_team_scores, game_player_scores = game_boxscore_parser(request_to_nba_feed(get_box_score_for_game_url(final_game_id)))
    all_game_team_scores.extend(game_team_scores)
    all_game_player_scores.extend(game_player_scores)

1 from 752
2 from 752
3 from 752
4 from 752
5 from 752
6 from 752
7 from 752
8 from 752
9 from 752
10 from 752
11 from 752
12 from 752
13 from 752
14 from 752
15 from 752
16 from 752
17 from 752
18 from 752
19 from 752
20 from 752
21 from 752
22 from 752
23 from 752
24 from 752
25 from 752
26 from 752
27 from 752
28 from 752
29 from 752
30 from 752
31 from 752
32 from 752
33 from 752
34 from 752
35 from 752
36 from 752
37 from 752
38 from 752
39 from 752
40 from 752
41 from 752
42 from 752
43 from 752
44 from 752
45 from 752
46 from 752
47 from 752
48 from 752
49 from 752
50 from 752
51 from 752
52 from 752
53 from 752
54 from 752
55 from 752
56 from 752
57 from 752
58 from 752
59 from 752
60 from 752
61 from 752
62 from 752
63 from 752
64 from 752
65 from 752
66 from 752
67 from 752
68 from 752
69 from 752
70 from 752
71 from 752
72 from 752
73 from 752
74 from 752
75 from 752
76 from 752
77 from 752
78 from 752
79 from 752
80 from 752
81 from 752
82 from 752
83 from 752
84 from 752
8

640 from 752
641 from 752
642 from 752
643 from 752
644 from 752
645 from 752
646 from 752
647 from 752
648 from 752
649 from 752
650 from 752
651 from 752
652 from 752
653 from 752
654 from 752
655 from 752
656 from 752
657 from 752
658 from 752
659 from 752
660 from 752
661 from 752
662 from 752
663 from 752
664 from 752
665 from 752
666 from 752
667 from 752
668 from 752
669 from 752
670 from 752
671 from 752
672 from 752
673 from 752
674 from 752
675 from 752
676 from 752
677 from 752
678 from 752
679 from 752
680 from 752
681 from 752
682 from 752
683 from 752
684 from 752
685 from 752
686 from 752
687 from 752
688 from 752
689 from 752
690 from 752
691 from 752
692 from 752
693 from 752
694 from 752
695 from 752
696 from 752
697 from 752
698 from 752
699 from 752
700 from 752
701 from 752
702 from 752
703 from 752
704 from 752
705 from 752
706 from 752
707 from 752
708 from 752
709 from 752
710 from 752
711 from 752
712 from 752
713 from 752
714 from 752
715 from 752
716 from 752

# Save to csv files

In [28]:
save_into_csv('Teams', all_teams)
save_into_csv('Players', all_players)
save_into_csv('Games', all_games)
save_into_csv('TeamRosters', all_team_rosters)

save_into_csv('GameLineUp', all_game_lineups)
save_into_csv('GameTeamScore', all_game_team_scores)
save_into_csv('GamePlayerScore', all_game_player_scores)

In [29]:
for team_score in all_game_team_scores:
    print(team_score)

TeamID: 1610612758, Q1: 26, Q2: 32, Q3: 29, Q4: 21, GameID: 0022200547, FGA: 96, FGM: 39, FGPER: 40, ThreePM: 15, ThreePA: 43, ThreePPER: 34, FTM: 15, FTA: 15, FTPER: 100, OREB: 14, DREB: 33, AST: 24, STL: 7, BLK: 4, TournOvers: 15, PTS: 108
TeamID: 1610612763, Q1: 27, Q2: 32, Q3: 32, Q4: 27, GameID: 0022200547, FGA: 100, FGM: 46, FGPER: 46, ThreePM: 10, ThreePA: 32, ThreePPER: 31, FTM: 16, FTA: 24, FTPER: 66, OREB: 21, DREB: 36, AST: 25, STL: 11, BLK: 8, TournOvers: 11, PTS: 118
TeamID: 1610612764, Q1: 34, Q2: 34, Q3: 20, Q4: 30, GameID: 0022200548, FGA: 94, FGM: 50, FGPER: 53, ThreePM: 7, ThreePA: 27, ThreePPER: 25, FTM: 11, FTA: 16, FTPER: 68, OREB: 12, DREB: 42, AST: 29, STL: 9, BLK: 6, TournOvers: 17, PTS: 118
TeamID: 1610612749, Q1: 17, Q2: 33, Q3: 20, Q4: 25, GameID: 0022200548, FGA: 96, FGM: 34, FGPER: 35, ThreePM: 13, ThreePA: 49, ThreePPER: 26, FTM: 14, FTA: 15, FTPER: 93, OREB: 14, DREB: 30, AST: 22, STL: 11, BLK: 3, TournOvers: 16, PTS: 95
TeamID: 1610612738, Q1: 28, Q2: 26

# SQL Section:

In [46]:
# Connect to the DB:
database_name = 'Test_20.1.db'
conn = sqlite3.connect(database_name)
print ("Opened database successfully")

Opened database successfully


## Table Creation:

In [47]:
# Create SQL tables:
conn.executescript(f'''

CREATE TABLE Teams (
    ID BingInt PRIMARY KEY,
    TeamName VARCHAR(100) NOT NULL,
    TeamCode VARCHAR(15) NOT NULL,
    City VARCHAR(100) NOT NULL
);

CREATE TABLE Players (
    ID BingInt PRIMARY KEY,
    FirstName VARCHAR(100) NOT NULL,
    LastName VARCHAR(100) NOT NULL,
    DateOfBirth DATE NOT NULL,
    Position VARCHAR(100) NOT NULL,
    Height VARCHAR(100),
    Weight VARCHAR(100),
    YearsInLeague INT,
    Country VARCHAR(100)
);

CREATE TABLE TeamRosters (
    TeamID BingInt NOT NULL,
    PlayerID BingInt NOT NULL
);


CREATE TABLE Games (
    ID BingInt PRIMARY KEY,
    Date DATE NOT NULL,
    HomeTeamID BingInt NOT NULL,
    VisitorTeamID BingInt NOT NULL,
    LiveStatus VARCHAR(100) NOT NULL
);

CREATE TABLE GameLineUps (
    GameID BingInt NOT NULL,
    TeamID BingInt NOT NULL,
    PlayerID BingInt NOT NULL
);

CREATE TABLE GameTeamScores (
    GameID BingInt NOT NULL,
    TeamID BingInt NOT NULL,
    ScoreQOne INT,
    ScoreQTwo INT,
    ScoreQThree INT,
    ScoreQFour INT,
    FGA INT,
    FGM INT,
    FGPER FLOAT,
    ThreePA INT,
    ThreePM INT,
    ThreePPER FLOAT,
    FTA INT,
    FTM INT,
    FTPER FLOAT,
    OREB INT,
    DREB INT,
    AST INT,
    STL INT,
    BLK INT,
    TournOvers INT,
    PTS INT
);

CREATE TABLE GamePlayerScores (
    GameID BingInt NOT NULL,
    PlayerID BingInt NOT NULL,
    PlayTimeInSec INT,
    PlusMinusRank INT,
    FGA INT,
    FGM INT,
    FGPER FLOAT,
    ThreePA INT,
    ThreePM INT,
    ThreePPER FLOAT,
    FTA INT,
    FTM INT,
    FTPER FLOAT,
    OREB INT,
    DREB INT,
    AST INT,
    STL INT,
    BLK INT,
    TournOvers INT,
    PTS INT
);

''')
print("Tables created successfully")

Tables created successfully


## Insert Functions:

In [120]:
# Get property names of an object class
def get_columns_from_prop(element):
    # Get the properties of the class
    properties = vars(element)
    
     # Filter out the built-in properties and methods
    properties = [f"'{prop}'" for prop in properties.keys() if not prop.startswith("__")]
    
    # Concatenate the properties into a single string, separated by a comma
    property_string = ",".join(properties)
    
    return property_string

# Get values of an object class
def get_values_from_element(element):
    slash = '\''
    
    # Get the values of the properties of the object
    values = [str(value) if not isinstance(value, str) else f"'{value.replace(slash, '')}'" for value in vars(element).values()]

    # Concatenate the values into a single string, separated by a comma, replace None with empty str
    value_string = (",".join(values)).replace('None', '\'\'')
    
    return value_string

# Insert elements
def insert_elements(elements, table_name):
    
    if(len(elements) > 0):
        #Extract the name of the properties for the column name.
        columns = get_columns_from_prop(elements[0])
        
        total_num = len(elements)
        counter = 0
        #Connect to the db.
        with sqlite3.connect(database_name) as conn:
            queries = ''
            for element in elements:
                try:                    
                    print(f'''{counter} from {total_num}''')
                    counter += 1
                    # Execute and commit the command
                    conn.execute(f'''
                                INSERT INTO {table_name} ({columns})
                                VALUES ({get_values_from_element(element)})''')
                    conn.commit()
                except Exception as e:
                    print(f'Error: {e}')        

In [107]:
#Insert Teams
insert_elements(all_teams, 'Teams')


                                INSERT INTO Teams ('ID','TeamName','TeamCode','City')
                                VALUES (1610612766,'Hornets','CHA','Charlotte')

                                INSERT INTO Teams ('ID','TeamName','TeamCode','City')
                                VALUES (1610612738,'Celtics','BOS','Boston')

                                INSERT INTO Teams ('ID','TeamName','TeamCode','City')
                                VALUES (1610612752,'Knicks','NYK','New York')

                                INSERT INTO Teams ('ID','TeamName','TeamCode','City')
                                VALUES (1610612757,'Trail Blazers','POR','Portland')

                                INSERT INTO Teams ('ID','TeamName','TeamCode','City')
                                VALUES (1610612750,'Timberwolves','MIN','Minnesota')

                                INSERT INTO Teams ('ID','TeamName','TeamCode','City')
                                VALUES (1610612741,'Bulls','CHI','Chicago

In [ ]:
#Insert Players
insert_elements(all_players, 'Players')

In [108]:
#Insert TeamRosters
insert_elements(all_team_rosters, 'TeamRosters')


                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612737,203992)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612737,1629717)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612737,1630602)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612737,203991)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612737,1628381)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612737,1630536)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612737,1629633)

                            


                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612739,1629712)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612739,201567)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612739,1628374)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612739,1628378)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612739,1630596)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612739,1630600)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612739,1630612)

                           


                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612742,1630267)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612742,1627737)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612742,203915)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612742,1629029)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612742,1628416)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612742,1627827)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612742,1630182)

                           


                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612744,202691)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612744,1629308)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612744,1629683)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612744,203952)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612744,1630652)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612744,1630164)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612745,1628425)

                            


                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612747,1629117)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612747,1631289)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612747,1627885)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612747,1629659)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612747,2730)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612747,1630643)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612747,2544)

                                

...


                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612757,1629642)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612757,1629712)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612757,203463)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612757,1630627)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612757,203994)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612757,1627780)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612757,203658)

                             


                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612759,1627749)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612759,1627751)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612759,1630563)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612759,1626196)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612759,1629676)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612759,1631110)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612759,1630597)

                          


                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612761,1630551)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612761,1630288)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612761,1630201)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612761,1631199)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612761,1641501)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612761,1627823)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612761,1628367)

                          


                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612763,1630217)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612763,1631368)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612763,1628415)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612763,1631113)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612763,1629634)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612763,1629633)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612763,1631367)

                          


                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612765,1631105)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612765,1629035)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612765,1631266)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612765,1630568)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612765,203924)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612765,1630165)

                                INSERT INTO TeamRosters ('TeamID','PlayerID')
                                VALUES (1610612765,1631093)

                           

In [110]:
#Insert Games
insert_elements(all_games, 'Games')

In [119]:
#Insert LineUps
insert_elements(all_game_lineups, 'GameLineUps')

In [135]:
#Insert TeamScores
insert_elements(all_game_team_scores, 'GameTeamScores')

0 from 1504
1 from 1504
2 from 1504
3 from 1504
4 from 1504
5 from 1504
6 from 1504
7 from 1504
8 from 1504
9 from 1504
10 from 1504
11 from 1504
12 from 1504
13 from 1504
14 from 1504
15 from 1504
16 from 1504
17 from 1504
18 from 1504
19 from 1504
20 from 1504
21 from 1504
22 from 1504
23 from 1504
24 from 1504
25 from 1504
26 from 1504
27 from 1504
28 from 1504
29 from 1504
30 from 1504
31 from 1504
32 from 1504
33 from 1504
34 from 1504
35 from 1504
36 from 1504
37 from 1504
38 from 1504
39 from 1504
40 from 1504
41 from 1504
42 from 1504
43 from 1504
44 from 1504
45 from 1504
46 from 1504
47 from 1504
48 from 1504
49 from 1504
50 from 1504
51 from 1504
52 from 1504
53 from 1504
54 from 1504
55 from 1504
56 from 1504
57 from 1504
58 from 1504
59 from 1504
60 from 1504
61 from 1504
62 from 1504
63 from 1504
64 from 1504
65 from 1504
66 from 1504
67 from 1504
68 from 1504
69 from 1504
70 from 1504
71 from 1504
72 from 1504
73 from 1504
74 from 1504
75 from 1504
76 from 1504
77 from 1

618 from 1504
619 from 1504
620 from 1504
621 from 1504
622 from 1504
623 from 1504
624 from 1504
625 from 1504
626 from 1504
627 from 1504
628 from 1504
629 from 1504
630 from 1504
631 from 1504
632 from 1504
633 from 1504
634 from 1504
635 from 1504
636 from 1504
637 from 1504
638 from 1504
639 from 1504
640 from 1504
641 from 1504
642 from 1504
643 from 1504
644 from 1504
645 from 1504
646 from 1504
647 from 1504
648 from 1504
649 from 1504
650 from 1504
651 from 1504
652 from 1504
653 from 1504
654 from 1504
655 from 1504
656 from 1504
657 from 1504
658 from 1504
659 from 1504
660 from 1504
661 from 1504
662 from 1504
663 from 1504
664 from 1504
665 from 1504
666 from 1504
667 from 1504
668 from 1504
669 from 1504
670 from 1504
671 from 1504
672 from 1504
673 from 1504
674 from 1504
675 from 1504
676 from 1504
677 from 1504
678 from 1504
679 from 1504
680 from 1504
681 from 1504
682 from 1504
683 from 1504
684 from 1504
685 from 1504
686 from 1504
687 from 1504
688 from 1504
689 fr

1194 from 1504
1195 from 1504
1196 from 1504
1197 from 1504
1198 from 1504
1199 from 1504
1200 from 1504
1201 from 1504
1202 from 1504
1203 from 1504
1204 from 1504
1205 from 1504
1206 from 1504
1207 from 1504
1208 from 1504
1209 from 1504
1210 from 1504
1211 from 1504
1212 from 1504
1213 from 1504
1214 from 1504
1215 from 1504
1216 from 1504
1217 from 1504
1218 from 1504
1219 from 1504
1220 from 1504
1221 from 1504
1222 from 1504
1223 from 1504
1224 from 1504
1225 from 1504
1226 from 1504
1227 from 1504
1228 from 1504
1229 from 1504
1230 from 1504
1231 from 1504
1232 from 1504
1233 from 1504
1234 from 1504
1235 from 1504
1236 from 1504
1237 from 1504
1238 from 1504
1239 from 1504
1240 from 1504
1241 from 1504
1242 from 1504
1243 from 1504
1244 from 1504
1245 from 1504
1246 from 1504
1247 from 1504
1248 from 1504
1249 from 1504
1250 from 1504
1251 from 1504
1252 from 1504
1253 from 1504
1254 from 1504
1255 from 1504
1256 from 1504
1257 from 1504
1258 from 1504
1259 from 1504
1260 from 

In [136]:
#Insert PlayerScores
insert_elements(all_game_player_scores, 'GamePlayerScores')

0 from 25453
1 from 25453
2 from 25453
3 from 25453
4 from 25453
5 from 25453
6 from 25453
7 from 25453
8 from 25453
9 from 25453
10 from 25453
11 from 25453
12 from 25453
13 from 25453
14 from 25453
15 from 25453
16 from 25453
17 from 25453
18 from 25453
19 from 25453
20 from 25453
21 from 25453
22 from 25453
23 from 25453
24 from 25453
25 from 25453
26 from 25453
27 from 25453
28 from 25453
29 from 25453
30 from 25453
31 from 25453
32 from 25453
33 from 25453
34 from 25453
35 from 25453
36 from 25453
37 from 25453
38 from 25453
39 from 25453
40 from 25453
41 from 25453
42 from 25453
43 from 25453
44 from 25453
45 from 25453
46 from 25453
47 from 25453
48 from 25453
49 from 25453
50 from 25453
51 from 25453
52 from 25453
53 from 25453
54 from 25453
55 from 25453
56 from 25453
57 from 25453
58 from 25453
59 from 25453
60 from 25453
61 from 25453
62 from 25453
63 from 25453
64 from 25453
65 from 25453
66 from 25453
67 from 25453
68 from 25453
69 from 25453
70 from 25453
71 from 25453
72

564 from 25453
565 from 25453
566 from 25453
567 from 25453
568 from 25453
569 from 25453
570 from 25453
571 from 25453
572 from 25453
573 from 25453
574 from 25453
575 from 25453
576 from 25453
577 from 25453
578 from 25453
579 from 25453
580 from 25453
581 from 25453
582 from 25453
583 from 25453
584 from 25453
585 from 25453
586 from 25453
587 from 25453
588 from 25453
589 from 25453
590 from 25453
591 from 25453
592 from 25453
593 from 25453
594 from 25453
595 from 25453
596 from 25453
597 from 25453
598 from 25453
599 from 25453
600 from 25453
601 from 25453
602 from 25453
603 from 25453
604 from 25453
605 from 25453
606 from 25453
607 from 25453
608 from 25453
609 from 25453
610 from 25453
611 from 25453
612 from 25453
613 from 25453
614 from 25453
615 from 25453
616 from 25453
617 from 25453
618 from 25453
619 from 25453
620 from 25453
621 from 25453
622 from 25453
623 from 25453
624 from 25453
625 from 25453
626 from 25453
627 from 25453
628 from 25453
629 from 25453
630 from 2

1119 from 25453
1120 from 25453
1121 from 25453
1122 from 25453
1123 from 25453
1124 from 25453
1125 from 25453
1126 from 25453
1127 from 25453
1128 from 25453
1129 from 25453
1130 from 25453
1131 from 25453
1132 from 25453
1133 from 25453
1134 from 25453
1135 from 25453
1136 from 25453
1137 from 25453
1138 from 25453
1139 from 25453
1140 from 25453
1141 from 25453
1142 from 25453
1143 from 25453
1144 from 25453
1145 from 25453
1146 from 25453
1147 from 25453
1148 from 25453
1149 from 25453
1150 from 25453
1151 from 25453
1152 from 25453
1153 from 25453
1154 from 25453
1155 from 25453
1156 from 25453
1157 from 25453
1158 from 25453
1159 from 25453
1160 from 25453
1161 from 25453
1162 from 25453
1163 from 25453
1164 from 25453
1165 from 25453
1166 from 25453
1167 from 25453
1168 from 25453
1169 from 25453
1170 from 25453
1171 from 25453
1172 from 25453
1173 from 25453
1174 from 25453
1175 from 25453
1176 from 25453
1177 from 25453
1178 from 25453
1179 from 25453
1180 from 25453
1181 fro

1642 from 25453
1643 from 25453
1644 from 25453
1645 from 25453
1646 from 25453
1647 from 25453
1648 from 25453
1649 from 25453
1650 from 25453
1651 from 25453
1652 from 25453
1653 from 25453
1654 from 25453
1655 from 25453
1656 from 25453
1657 from 25453
1658 from 25453
1659 from 25453
1660 from 25453
1661 from 25453
1662 from 25453
1663 from 25453
1664 from 25453
1665 from 25453
1666 from 25453
1667 from 25453
1668 from 25453
1669 from 25453
1670 from 25453
1671 from 25453
1672 from 25453
1673 from 25453
1674 from 25453
1675 from 25453
1676 from 25453
1677 from 25453
1678 from 25453
1679 from 25453
1680 from 25453
1681 from 25453
1682 from 25453
1683 from 25453
1684 from 25453
1685 from 25453
1686 from 25453
1687 from 25453
1688 from 25453
1689 from 25453
1690 from 25453
1691 from 25453
1692 from 25453
1693 from 25453
1694 from 25453
1695 from 25453
1696 from 25453
1697 from 25453
1698 from 25453
1699 from 25453
1700 from 25453
1701 from 25453
1702 from 25453
1703 from 25453
1704 fro

2162 from 25453
2163 from 25453
2164 from 25453
2165 from 25453
2166 from 25453
2167 from 25453
2168 from 25453
2169 from 25453
2170 from 25453
2171 from 25453
2172 from 25453
2173 from 25453
2174 from 25453
2175 from 25453
2176 from 25453
2177 from 25453
2178 from 25453
2179 from 25453
2180 from 25453
2181 from 25453
2182 from 25453
2183 from 25453
2184 from 25453
2185 from 25453
2186 from 25453
2187 from 25453
2188 from 25453
2189 from 25453
2190 from 25453
2191 from 25453
2192 from 25453
2193 from 25453
2194 from 25453
2195 from 25453
2196 from 25453
2197 from 25453
2198 from 25453
2199 from 25453
2200 from 25453
2201 from 25453
2202 from 25453
2203 from 25453
2204 from 25453
2205 from 25453
2206 from 25453
2207 from 25453
2208 from 25453
2209 from 25453
2210 from 25453
2211 from 25453
2212 from 25453
2213 from 25453
2214 from 25453
2215 from 25453
2216 from 25453
2217 from 25453
2218 from 25453
2219 from 25453
2220 from 25453
2221 from 25453
2222 from 25453
2223 from 25453
2224 fro

...

23207 from 25453
23208 from 25453
23209 from 25453
23210 from 25453
23211 from 25453
23212 from 25453
23213 from 25453
23214 from 25453
23215 from 25453
23216 from 25453
23217 from 25453
23218 from 25453
23219 from 25453
23220 from 25453
23221 from 25453
23222 from 25453
23223 from 25453
23224 from 25453
23225 from 25453
23226 from 25453
23227 from 25453
23228 from 25453
23229 from 25453
23230 from 25453
23231 from 25453
23232 from 25453
23233 from 25453
23234 from 25453
23235 from 25453
23236 from 25453
23237 from 25453
23238 from 25453
23239 from 25453
23240 from 25453
23241 from 25453
23242 from 25453
23243 from 25453
23244 from 25453
23245 from 25453
23246 from 25453
23247 from 25453
23248 from 25453
23249 from 25453
23250 from 25453
23251 from 25453
23252 from 25453
23253 from 25453
23254 from 25453
23255 from 25453
23256 from 25453
23257 from 25453
23258 from 25453
23259 from 25453
23260 from 25453
23261 from 25453
23262 from 25453
23263 from 25453
23264 from 25453
23265 from 254

23702 from 25453
23703 from 25453
23704 from 25453
23705 from 25453
23706 from 25453
23707 from 25453
23708 from 25453
23709 from 25453
23710 from 25453
23711 from 25453
23712 from 25453
23713 from 25453
23714 from 25453
23715 from 25453
23716 from 25453
23717 from 25453
23718 from 25453
23719 from 25453
23720 from 25453
23721 from 25453
23722 from 25453
23723 from 25453
23724 from 25453
23725 from 25453
23726 from 25453
23727 from 25453
23728 from 25453
23729 from 25453
23730 from 25453
23731 from 25453
23732 from 25453
23733 from 25453
23734 from 25453
23735 from 25453
23736 from 25453
23737 from 25453
23738 from 25453
23739 from 25453
23740 from 25453
23741 from 25453
23742 from 25453
23743 from 25453
23744 from 25453
23745 from 25453
23746 from 25453
23747 from 25453
23748 from 25453
23749 from 25453
23750 from 25453
23751 from 25453
23752 from 25453
23753 from 25453
23754 from 25453
23755 from 25453
23756 from 25453
23757 from 25453
23758 from 25453
23759 from 25453
23760 from 254

24187 from 25453
24188 from 25453
24189 from 25453
24190 from 25453
24191 from 25453
24192 from 25453
24193 from 25453
24194 from 25453
24195 from 25453
24196 from 25453
24197 from 25453
24198 from 25453
24199 from 25453
24200 from 25453
24201 from 25453
24202 from 25453
24203 from 25453
24204 from 25453
24205 from 25453
24206 from 25453
24207 from 25453
24208 from 25453
24209 from 25453
24210 from 25453
24211 from 25453
24212 from 25453
24213 from 25453
24214 from 25453
24215 from 25453
24216 from 25453
24217 from 25453
24218 from 25453
24219 from 25453
24220 from 25453
24221 from 25453
24222 from 25453
24223 from 25453
24224 from 25453
24225 from 25453
24226 from 25453
24227 from 25453
24228 from 25453
24229 from 25453
24230 from 25453
24231 from 25453
24232 from 25453
24233 from 25453
24234 from 25453
24235 from 25453
24236 from 25453
24237 from 25453
24238 from 25453
24239 from 25453
24240 from 25453
24241 from 25453
24242 from 25453
24243 from 25453
24244 from 25453
24245 from 254

24674 from 25453
24675 from 25453
24676 from 25453
24677 from 25453
24678 from 25453
24679 from 25453
24680 from 25453
24681 from 25453
24682 from 25453
24683 from 25453
24684 from 25453
24685 from 25453
24686 from 25453
24687 from 25453
24688 from 25453
24689 from 25453
24690 from 25453
24691 from 25453
24692 from 25453
24693 from 25453
24694 from 25453
24695 from 25453
24696 from 25453
24697 from 25453
24698 from 25453
24699 from 25453
24700 from 25453
24701 from 25453
24702 from 25453
24703 from 25453
24704 from 25453
24705 from 25453
24706 from 25453
24707 from 25453
24708 from 25453
24709 from 25453
24710 from 25453
24711 from 25453
24712 from 25453
24713 from 25453
24714 from 25453
24715 from 25453
24716 from 25453
24717 from 25453
24718 from 25453
24719 from 25453
24720 from 25453
24721 from 25453
24722 from 25453
24723 from 25453
24724 from 25453
24725 from 25453
24726 from 25453
24727 from 25453
24728 from 25453
24729 from 25453
24730 from 25453
24731 from 25453
24732 from 254

25163 from 25453
25164 from 25453
25165 from 25453
25166 from 25453
25167 from 25453
25168 from 25453
25169 from 25453
25170 from 25453
25171 from 25453
25172 from 25453
25173 from 25453
25174 from 25453
25175 from 25453
25176 from 25453
25177 from 25453
25178 from 25453
25179 from 25453
25180 from 25453
25181 from 25453
25182 from 25453
25183 from 25453
25184 from 25453
25185 from 25453
25186 from 25453
25187 from 25453
25188 from 25453
25189 from 25453
25190 from 25453
25191 from 25453
25192 from 25453
25193 from 25453
25194 from 25453
25195 from 25453
25196 from 25453
25197 from 25453
25198 from 25453
25199 from 25453
25200 from 25453
25201 from 25453
25202 from 25453
25203 from 25453
25204 from 25453
25205 from 25453
25206 from 25453
25207 from 25453
25208 from 25453
25209 from 25453
25210 from 25453
25211 from 25453
25212 from 25453
25213 from 25453
25214 from 25453
25215 from 25453
25216 from 25453
25217 from 25453
25218 from 25453
25219 from 25453
25220 from 25453
25221 from 254

## Getters:

In [111]:
# Get Teams:
def get_teams():    
    teams = []
    conn = sqlite3.connect(database_name)
    cursor = conn.execute(f'''select * from Teams''')

    for row in cursor:
        ID, TeamName, TeamCode, City = row
        team = Team(ID, TeamName, TeamCode, City)
        teams.append(team)
    
    #close connection
    conn.close()  
    
    return teams

In [ ]:
#Get Players:
def get_players():    
    players = []
    conn = sqlite3.connect(database_name)
    cursor = conn.execute(f'''select * from Players''')

    for row in cursor:
        ID, FirstName, LastName, DateOfBirth, Position, Height, Weight, YearsInLeague, Country = row
        player = Player(ID, FirstName, LastName, DateOfBirth, Position, Height, Weight, YearsInLeague, Country)
        players.append(player)
    
    #close connection
    conn.close()  
    
    return players

In [115]:
#Get Games:
def get_games():    
    games = []
    conn = sqlite3.connect(database_name)
    cursor = conn.execute(f'''select * from Games''')

    for row in cursor:
        ID, Date, HomeTeamID, VisitorTeamID, LiveStatus = row
        game = Game(ID, Date, HomeTeamID, VisitorTeamID, LiveStatus)
        games.append(game)
    
    #close connection
    conn.close()  
    
    return games

In [117]:
#Get TeamRosters:
def get_team_rosters():    
    rosters = []
    conn = sqlite3.connect(database_name)
    cursor = conn.execute(f'''select * from TeamRosters''')

    for row in cursor:
        TeamID, PlayerID = row
        roster = TeamRoster(TeamID, PlayerID)
        rosters.append(roster)
    
    #close connection
    conn.close()  
    
    return rosters        

In [126]:
#Get GameLineUp:
def get_all_game_lineups():    
    lineups = []
    conn = sqlite3.connect(database_name)
    cursor = conn.execute(f'''select * from GameLineUps''')

    for row in cursor:
        GameID, TeamID, PlayerID = row
        lineup = GameLineUp(GameID, TeamID, PlayerID)
        lineups.append(lineup)
    
    #close connection
    conn.close()  
    
    return lineups      

def get_game_lineups(game_id):    
    lineups = []
    conn = sqlite3.connect(database_name)
    cursor = conn.execute(f'''select * from GameLineUps where GameId = {game_id}''')

    for row in cursor:
        GameID, TeamID, PlayerID = row
        lineup = GameLineUp(GameID, TeamID, PlayerID)
        lineups.append(lineup)
    
    #close connection
    conn.close()  
    
    return lineups     

In [129]:
#Get Player Scores:
def get_all_player_scores():    
    player_scores = []
    conn = sqlite3.connect(database_name)
    cursor = conn.execute(f'''select * from GamePlayerScores''')

    for row in cursor:
        GameID, PlayerID, PlayTimeInSec, PlusMinusRank, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TournOvers, PTS = row
        player_score = GamePlayerScores(GameID, PlayerID, PlayTimeInSec, PlusMinusRank, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TournOvers, PTS)
        player_scores.append(player_score)
    
    #close connection
    conn.close()  
    
    return player_scores   

def get_player_scores(game_id):    
    player_scores = []
    conn = sqlite3.connect(database_name)
    cursor = conn.execute(f'''select * from GamePlayerScores where GameId = {game_id}''')

    for row in cursor:
        GameID, PlayerID, PlayTimeInSec, PlusMinusRank, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TournOvers, PTS = row
        player_score = GamePlayerScores(GameID, PlayerID, PlayTimeInSec, PlusMinusRank, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TournOvers, PTS)
        player_scores.append(player_score)
    
    #close connection
    conn.close()  
    
    return player_scores   

In [144]:
#Get Team Scores:
def get_all_team_scores():    
    team_scores = []
    conn = sqlite3.connect(database_name)
    cursor = conn.execute(f'''select * from GameTeamScores''')

    for row in cursor:
        GameID, TeamID, ScoreQOne, ScoreQTwo, ScoreQTwo, ScoreQThree, ScoreQFour, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TournOvers, PTS = row
        team_score = GameTeamScore(GameID, TeamID, ScoreQOne, ScoreQTwo, ScoreQTwo, ScoreQThree, ScoreQFour, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TournOvers, PTS)
        team_scores.append(team_score)
    
    #close connection
    conn.close()  
    
    return team_scores  

def get_team_scores(game_id):    
    team_scores = []
    conn = sqlite3.connect(database_name)
    cursor = conn.execute(f'''select * from GameTeamScores where GameId = {game_id}''')

    for row in cursor:
        GameID, TeamID, ScoreQOne, ScoreQTwo, ScoreQThree, ScoreQFour, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TournOvers, PTS = row
        team_score = GameTeamScore(GameID, TeamID, ScoreQOne, ScoreQTwo, ScoreQThree, ScoreQFour, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TournOvers, PTS)
        team_scores.append(team_score)
    
    #close connection
    conn.close()  
    
    return team_scores 

In [145]:
elements = get_team_scores(game_id)

for element in elements:
    print(element)

TeamID: 1610612766, Q1: 24, Q2: 22, Q3: 31, Q4: 16, GameID: 12200005, FGA: 101, FGM: 35, FGPER: 34.0, ThreePM: 5, ThreePA: 33, ThreePPER: 15.0, FTM: 18, FTA: 28, FTPER: 64.0, OREB: 16, DREB: 27, AST: 13, STL: 13, BLK: 2, TournOvers: 13, PTS: 93
TeamID: 1610612738, Q1: 29, Q2: 39, Q3: 39, Q4: 27, GameID: 12200005, FGA: 84, FGM: 48, FGPER: 57.0, ThreePM: 22, ThreePA: 47, ThreePPER: 46.0, FTM: 16, FTA: 20, FTPER: 80.0, OREB: 8, DREB: 48, AST: 41, STL: 4, BLK: 3, TournOvers: 23, PTS: 134


In [102]:
#Delete the tables
#conn = sqlite3.connect(database_name)
#conn.execute(f'''DELETE from Players''')
#conn.commit()
#close connection
#conn.close()  

In [54]:
#Close the connection
conn.close()